In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# LOADS 3 FILES
#     1. 311_06.csv -> contains 311 Public complaints data from New York City
#     2. CrimeData.csv -> Multi-year crime records
#     3. Regions.csv -> Geographical divisions of the city into precincts

import pandas as pd
import numpy as np
import shapely.wkt
from shapely.geometry import Point, Polygon, MultiPolygon

# Change the 'year' and file locations according to your dataset
year = '2006'
file311 = "/content/gdrive/MyDrive/Dataset/311_06.csv"
fileCrime = "/content/gdrive/MyDrive/Dataset/CrimeData.csv"
fileRegions = "/content/gdrive/MyDrive/Dataset/Regions.csv"

# Extract:
#     1. Relevant columns
#     2. Relevant types of complaints, from the 'Complaint Type' column

data311 = pd.read_csv(file311,low_memory=False)
relevantColumns311 = ['Created Date','Latitude','Longitude','Complaint Type']
relevantComplaints311 = ['Blocked Driveway','Building/Use','Noise','Safety']
finalColumns311 = ['Created Date','Complaint Type','Precincts']
data311 = data311[relevantColumns311]
data311 = data311.loc[(data311['Complaint Type'] == 'Blocked Driveway') | (data311['Complaint Type'] == 'Building/Use') | (data311['Complaint Type'] == 'Noise') | (data311['Complaint Type'] == 'Safety')]
print('Anomalies preprocessed!!!')

# Extract:
#     1. Relevant columns
#     2. Relevant types of crimes, from the 'OFNS_DESC' column

crimeData = pd.read_csv(fileCrime, low_memory=False)
relevantColumnsCR = ['CMPLNT_TO_DT','CMPLNT_TO_TM','CMPLNT_FR_DT','RPT_DT','Lat_Lon','Latitude','Longitude','OFNS_DESC']
finalColumnsCR = ['Date','OFNS_DESC','CMPLNT_TO_TM','Longitude','Latitude','Precincts']
relevantCrimesCR = ['ROBBERY', 'BURGLARY', 'FELONY ASSAULT','GRAND LARCENY']
locations = pd.read_csv("/content/gdrive/MyDrive/Dataset/Regions.csv", low_memory=False)
crime2006 = crimeData.loc[crimeData['RPT_DT'].str.endswith(year)]
crime2006 = crime2006[relevantColumnsCR]
crime2006 = crime2006.loc[(crime2006['OFNS_DESC'] == 'ROBBERY') | (crime2006['OFNS_DESC'] == 'BURGLARY') | (crime2006['OFNS_DESC'] == 'FELONY ASSAULT') | (crime2006['OFNS_DESC'] == 'GRAND LARCENY')]
print('Crime Preprocessed!!!')

# Load the regions and store a MultiPolygon object(corresponding to the geometry of the region) and the precinct number

locations = pd.read_csv(fileRegions, low_memory=False)
precincts = {}
for index, row in locations.iterrows():
  precincts[row['Precinct']] = shapely.wkt.loads(row['the_geom'])

print('Locations loaded!!!')

Anomalies preprocessed!!!
Crime Preprocessed!!!
Locations loaded!!!


In [ ]:
# Extracts the latitude/longitude information of each 311 complaint reported
# And assigns it to one of the precincts defined earlier in form of a MultiPolygon

pos = 0
prec = np.ndarray((data311.shape[0],))

for index,row in data311.iterrows():

  poo = Point(row['Longitude'],row['Latitude'])  
  for key,val in precincts.items():
    if poo.within(val):
      prec[pos] = key
      break

  pos=pos+1
  if(pos%1000 == 0):
    print ("processed "+str(pos)+" records!!")

# Adds the precincts information to the dataset
data311['Precincts'] = prec.astype(int)
print("Done!!")

# Extract the final columns needed for training the model
data311['Precincts'] = data311['Precincts'].astype(np.int64)
data311 = data311[data311['Precincts'] >= 0]
data311 = data311[finalColumns311]

print('Anomaly Preprocessing complete!!!')

processed 1000 records!!
processed 2000 records!!
processed 3000 records!!
processed 4000 records!!
processed 5000 records!!
processed 6000 records!!
processed 7000 records!!
processed 8000 records!!
processed 9000 records!!
processed 10000 records!!
processed 11000 records!!
processed 12000 records!!
processed 13000 records!!
processed 14000 records!!
processed 15000 records!!
processed 16000 records!!
processed 17000 records!!
processed 18000 records!!
processed 19000 records!!
processed 20000 records!!
processed 21000 records!!
processed 22000 records!!
processed 23000 records!!
processed 24000 records!!
processed 25000 records!!
processed 26000 records!!
processed 27000 records!!
processed 28000 records!!
processed 29000 records!!
processed 30000 records!!
processed 31000 records!!
processed 32000 records!!
processed 33000 records!!
processed 34000 records!!
processed 35000 records!!
processed 36000 records!!
processed 37000 records!!
processed 38000 records!!
processed 39000 recor

In [ ]:
# Extracts the latitude/longitude information of each crime reported
# And assigns it to one of the precincts defined earlier in form of a MultiPolygon

# Also, some preprocessing is done on the date columns as well
# To ensure that the final date column doesn't have NaN values

pos = 0
prec = np.ndarray((crime2006.shape[0],))
date = np.ndarray((crime2006.shape[0],)).astype(str)

for index,row in crime2006.iterrows():
  
  if not(str(row['CMPLNT_FR_DT']) == 'nan'):
    date[pos] = row['CMPLNT_FR_DT']
  elif not(str(row['CMPLNT_TO_DT']) == 'nan'):
    date[pos] = row['CMPLNT_TO_DT']
  else:
    date[pos] = row['RPT_DT']
  
  poo = Point(row['Longitude'],row['Latitude'])  
  for key,val in precincts.items():
    if poo.within(val):
      prec[pos] = key
      break

  pos=pos+1
  if(pos%1000 == 0):
    print ("processed "+str(pos)+" records!!")

# Adds the precincts information to the dataset
# Adds the dates information to the dataset

crime2006['Precincts'] = prec.astype(int)
crime2006['Date'] = date
crime2006 = crime2006[crime2006['Precincts'] >= 0]

print("Done!!")

# Extract the final columns needed for training the model
crime2006 = crime2006[finalColumnsCR]
crime2006.sort_values(['Precincts','Date','OFNS_DESC'],ascending=[True,True,True],inplace=True)
crime2006 = crime2006[crime2006['Date'].str.contains(year)]
crime2006['Precincts'] = crime2006['Precincts'].astype(np.int64)

print('Crime Preprocessing complete!!!')

processed 1000 records!!
processed 2000 records!!
processed 3000 records!!
processed 4000 records!!
processed 5000 records!!
processed 6000 records!!
processed 7000 records!!
processed 8000 records!!
processed 9000 records!!
processed 10000 records!!
processed 11000 records!!
processed 12000 records!!
processed 13000 records!!
processed 14000 records!!
processed 15000 records!!
processed 16000 records!!
processed 17000 records!!
processed 18000 records!!
processed 19000 records!!
processed 20000 records!!
processed 21000 records!!
processed 22000 records!!
processed 23000 records!!
processed 24000 records!!
processed 25000 records!!
processed 26000 records!!
processed 27000 records!!
processed 28000 records!!
processed 29000 records!!
processed 30000 records!!
processed 31000 records!!
processed 32000 records!!
processed 33000 records!!
processed 34000 records!!
processed 35000 records!!
processed 36000 records!!
processed 37000 records!!
processed 38000 records!!
processed 39000 recor

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Generates inverted dictionaries for
#     1. Date for the particular year, according to the 'year' var defined earlier
#     2. Complaints: according to the 4 categories that we are considering
#     3. Precincts: the grographical divisions of the city
#     4. Crimes: according to the 4 categories we are considering for prediction

# The final input that we feed to the Heirarchical LSTMs is in form of matrices
# These inverted indices help in conversion of the dataframe to thse matrices

daterange = pd.date_range(start='01/01/'+year,end='12/31/'+year)
daterange = daterange.strftime('%m/%d/%Y')
daterange = daterange.tolist()

inv_dict = {}
for i in range(len(daterange)):
  inv_dict[daterange[i]] = i

inv_complaints = {}
for i in range(len(relevantComplaints311)):
  inv_complaints[relevantComplaints311[i]] = i

n_precincts = data311['Precincts'].nunique()
uniq_precincts = data311['Precincts'].unique()
inv_prec = {}
for i in range(n_precincts):
  inv_prec[uniq_precincts[i]] = i

inv_crimes = {}
for i in range(len(relevantCrimesCR)):
  inv_crimes[relevantCrimesCR[i]] = i

In [ ]:
# Generate the matrices for Crime (365x78x4) and 311 Complaints (365x78x4)
# Here the 2nd dimension represents the precinct and is 78, instead of 77
# This is because all the complaints/crimes with no geographical location have been assigned 0th position
# This 0th position is later discarded during the training process, but can be used if one wishes to do so

# Each matrix represents crime/complaint occurence over different categories(4), over different localities(78) over a period of 1 year(365)


# Generating the crime records matrices
matricesCR = [np.zeros((n_precincts,4),dtype=np.int64) for x in range(365)]
exceptions = 0

for idx, row in crime2006.iterrows():
  try:
    id1 = inv_prec[row['Precincts']]
    id2 = inv_dict[row['Date']]
    id3 = inv_crimes[row['OFNS_DESC']]
    matricesCR[id2][id1][id3] = matricesCR[id2][id1][id3] + 1
  except:
    print("Exception!!!")
    print("Precincts",id1)
    print("Date",id2)
    print("Offense",id3)
    exceptions = exceptions + 1

print ('Created crime matrices!!!')

# Generating the 311 complaints records matrices
matrices311 = [np.zeros((n_precincts,4),dtype=np.int64) for x in range(365)]
exceptions = 0

for idx, row in data311.iterrows():
  try:
    id1 = inv_prec[row['Precincts']]
    id2 = inv_dict[str(row['Created Date']).split()[0]]
    id3 = inv_complaints[row['Complaint Type']]
    matrices311[id2][id1][id3] = matrices311[id2][id1][id3] + 1
  except:
    print("Exception!!!")
    print("Precincts",id1)
    print("Date",id2)
    print("Offense",id3)
    exceptions = exceptions + 1

print('Created anomaly matrices!!!')

Created crime matrices!!!
Created anomaly matrices!!!


In [ ]:
!mkdir "/content/gdrive/MyDrive/Dataset/generated"

In [ ]:
import pickle


# Dump the matrices using pickle, to a file
output311 = '/content/gdrive/MyDrive/Dataset/generated/matrices311'
outputCrime = '/content/gdrive/MyDrive/Dataset/generated/matricesCR'

file311 = open(output311,'wb')
pickle.dump(matrices311,file311)

fileCR = open(outputCrime,'wb')
pickle.dump(matricesCR,fileCR)

# To load these files, use:
# with open(filename,'rb') as toOpen:
#     data = pickle.load(toOpen)